### clone代码

In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git

### 安装特定依赖

In [ ]:
! cd modelscope-agent && pip install -r requirements.txt

### 本地配置

In [ ]:
import os
os.chdir('modelscope-agent/examples/llms/multi_llm_with_alpha_umi')

import sys
sys.path.append('../../../')

### 部署模型

In [ ]:
try:
    import vllm
except ImportError:
    raise ImportError("The vllm package is not installed. Please make sure GPU env is ready and refer to https://docs.vllm.ai/en/latest/getting_started/installation.html")
    
os.system("export VLLM_USE_MODELSCOPE=True")
os.system("python -m vllm.entrypoints.openai.api_server \
 --model=iic/alpha-umi-planner-7b \
 --revision=v1.0.0 --trust-remote-code \
 --port 8090 \
 --dtype float16 \
 --gpu-memory-utilization 0.3 > planner.log &")

os.system("python -m vllm.entrypoints.openai.api_server \
 --model=iic/alpha-umi-caller-7b \
 --revision=v1.0.0 --trust-remote-code \
 --port 8091 \
 --dtype float16 \
 --gpu-memory-utilization 0.3 > caller.log &")

os.system("python -m vllm.entrypoints.openai.api_server \
 --model=iic/alpha-umi-summarizer-7b \
 --revision=v1.0.0 --trust-remote-code \
 --port 8092 \
 --dtype float16 \
 --gpu-memory-utilization 0.3 > summarizer.log &")


### API_KEY管理

In [ ]:
import os

print('请输入DASHSCOPE_API_KEY')
os.environ['DASHSCOPE_API_KEY'] = input()
print('请输入ModelScope Token')
os.environ['MODELSCOPE_API_TOKEN'] = input()
print('请输入RapidAPI Token')
os.environ['RAPID_API_TOKEN'] = input()

### 构建agent

In [ ]:
from modelscope_agent.agents.alpha_umi import AlphaUmi


llm_configs = {
    'planner_llm_config': {
        'model': 'iic/alpha-umi-planner-7b',
        'model_server': 'openai',
        'api_base': 'http://localhost:8090/v1',
        'is_chat': False
    },
    'caller_llm_config': {
        'model': 'iic/alpha-umi-caller-7b',
        'model_server': 'openai',
        'api_base': 'http://localhost:8091/v1',
        'is_chat': False
    },
    'summarizer_llm_config': {
        'model': 'iic/alpha-umi-summarizer-7b',
        'model_server': 'openai',
        'api_base': 'http://localhost:8092/v1',
        'is_chat': False
    },
}



function_list = ["get_data_fact_for_numbers", "get_math_fact_for_numbers", "get_year_fact_for_numbers",
                    "listquotes_for_current_exchange",
                    "exchange_for_current_exchange"]


bot = AlphaUmi(
        function_list=function_list,
        llm_planner=llm_configs['planner_llm_config'],
        llm_caller=llm_configs['caller_llm_config'],
        llm_summarizer=llm_configs['summarizer_llm_config'],
    )



### 执行agent

In [ ]:
response = bot.run('how many CNY can I exchange for 3.5 US dollar? \
                   also, give me a special property about the number of CNY after exchange')


In [ ]:
text = ''
for chunk in response:
    text += chunk
print(text)